In [ ]:
!pip install torch_geometric

In [3]:
import torch
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid
import torch.nn.functional as F
import torch.nn as nn
from datetime import datetime

import itertools
import os
#from utils import *


os.environ["DGLBACKEND"] = "pytorch"

import time
import copy
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
import torch.nn.utils.prune as prune


In [4]:
import warnings
warnings.filterwarnings('ignore')
import sys, os
sys.path.append(os.path.abspath("../"))

import torch


In [5]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')


Processing...
Done!


In [6]:
def get_num_parameters(model: nn.Module, count_nonzero_only=False) -> int:
    """
    calculate the total number of parameters of model
    :param count_nonzero_only: only count nonzero weights
    """
    num_counted_elements = 0
    for param in model.parameters():
        if count_nonzero_only:
            num_counted_elements += param.count_nonzero()
        else:
            num_counted_elements += param.numel()
    return num_counted_elements


def get_model_size(model: nn.Module, data_width=32, count_nonzero_only=False) -> int:
    """
    calculate the model size in bits
    :param data_width: #bits per element
    :param count_nonzero_only: only count nonzero weights
    """
    return get_num_parameters(model, count_nonzero_only) * data_width

Byte = 8
KiB = 1024 * Byte
MiB = 1024 * KiB
GiB = 1024 * MiB



In [7]:


class GCN(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features, 16)
        self.conv2 = GCNConv(16, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)


model = GCN(dataset.num_features, dataset.num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)


In [8]:
def test():

    model.eval()
    logits, accs = model(dataset[0]), []
    for _, mask in dataset[0]( 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(dataset[0].y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

In [ ]:
for epoch in range(1, 100):
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model(dataset[0]), dataset[0].y).backward()
    optimizer.step()

    val_acc, test_acc = test()
    if epoch % 20 == 0:
        print(f'Epoch: {epoch:03d}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')


In [ ]:
 val_acc, test_acc = test()

0.9428571428571428 0.928 0.925


Epoch: 059, Train: 0.9848, Test: 0.9020
Epoch: 104, Train: 0.9830, Test: 0.9100

##  Measurement of avarage of accuracy, and time inference

In [9]:
import time
import statistics as stat


Eva_final=dict()


Base_model_Acc=[]
T_base_model=[]

In [10]:

for i in range(10):


        model = GCN(dataset.num_features, dataset.num_classes)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)


        for epoch in range(1, 100):
            model.train()
            optimizer.zero_grad()
            F.nll_loss(model(dataset[0]), dataset[0].y).backward()
            optimizer.step()

            val_acc, test_acc = test()
            if epoch % 20 == 0:
                    print(f'Epoch: {epoch:03d}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')


        start = time.time()
        train_acc, test_acc = test()
        end = time.time()
        t_inference= end-start

        print(f"Train Accuracy: {train_acc}, Test Accuracy: {test_acc}, Time inference:{t_inference}")


        Base_model_Acc.append(test_acc)
        T_base_model.append(t_inference)

Epoch: 020, Val: 0.9080, Test: 0.9120
Epoch: 040, Val: 0.9400, Test: 0.9290
Epoch: 060, Val: 0.9460, Test: 0.9380
Epoch: 080, Val: 0.9500, Test: 0.9460
Train Accuracy: 0.958, Test Accuracy: 0.952, Time inference:0.00552821159362793
Epoch: 020, Val: 0.9040, Test: 0.9070
Epoch: 040, Val: 0.9360, Test: 0.9280
Epoch: 060, Val: 0.9400, Test: 0.9350
Epoch: 080, Val: 0.9500, Test: 0.9530
Train Accuracy: 0.954, Test Accuracy: 0.952, Time inference:0.006078481674194336
Epoch: 020, Val: 0.9080, Test: 0.8980
Epoch: 040, Val: 0.9260, Test: 0.9250
Epoch: 060, Val: 0.9380, Test: 0.9310
Epoch: 080, Val: 0.9600, Test: 0.9450
Train Accuracy: 0.956, Test Accuracy: 0.943, Time inference:0.008569717407226562
Epoch: 020, Val: 0.8980, Test: 0.8950
Epoch: 040, Val: 0.9320, Test: 0.9230
Epoch: 060, Val: 0.9420, Test: 0.9360
Epoch: 080, Val: 0.9520, Test: 0.9450
Train Accuracy: 0.96, Test Accuracy: 0.951, Time inference:0.005553722381591797
Epoch: 020, Val: 0.9020, Test: 0.8860
Epoch: 040, Val: 0.9340, Test: 0

In [11]:
print(f'Base_model_Acc in ten time:')
print ("{:<10}|{:<10}|{:<10}|{:<10}|{:<10}|{:<10}|{:<10}|{:<10}|{:<15}|{:<20}"\
    .format(Base_model_Acc[0],Base_model_Acc[1],Base_model_Acc[2],\
        Base_model_Acc[3],Base_model_Acc[4],Base_model_Acc[5],\
        Base_model_Acc[6],Base_model_Acc[7],Base_model_Acc[8],Base_model_Acc[9]))
print ("-"*110)
print(f'Time Inference model in ten time:')
#print(T_base_model)
print ("{:<10}|{:<10}|{:<10}|{:<10}|{:<10}|{:<10}|{:<10}|{:<10}|{:<15}|{:<20}"\
    .format(T_base_model[0],T_base_model[1],T_base_model[2],\
        T_base_model[3],T_base_model[4],T_base_model[5],\
        T_base_model[6],T_base_model[7],T_base_model[8],T_base_model[9]))

Base_model_Auc in ten time:
0.952     |0.952     |0.943     |0.951     |0.952     |0.953     |0.953     |0.947     |0.953          |0.951               
--------------------------------------------------------------------------------------------------------------
Time Inference model in ten time:
0.00552821159362793|0.006078481674194336|0.008569717407226562|0.005553722381591797|0.005684375762939453|0.006142139434814453|0.005501747131347656|0.00612950325012207|0.005793571472167969|0.008510351181030273


In [13]:
import statistics as stat

base_model_accuracy_mean = stat.mean(Base_model_Acc)
base_model_accuracy_std =  stat.stdev(Base_model_Acc)
desc_auc = "{:.3f} ± {:.3f}".format(base_model_accuracy_mean,base_model_accuracy_std)
print(f"Auc: {desc_auc}"  )

Eva_final.update({'base model accuracy':float(format(base_model_accuracy_mean, '.4f'))})

t_base_model_mean =stat.mean(T_base_model)
t_base_model_std =stat.stdev(T_base_model)
desc_T = "{:.3f} ± {:.3f}".format(t_base_model_mean,t_base_model_std)

print(f"Time inference :{desc_T}")
Eva_final.update({'time inference of base model':float(format(t_base_model_mean, '.6f'))})

Auc: 0.951 ± 0.003
Time inference :0.006 ± 0.001


In [15]:
def get_num_parameters(model, count_nonzero_only=False) -> int:
    """
    calculate the total number of parameters of model
    :param count_nonzero_only: only count nonzero weights
    """
    num_counted_elements = 0
    for param in model.parameters():
        if count_nonzero_only:
            num_counted_elements += param.count_nonzero()
        else:
            num_counted_elements += param.numel()
    return num_counted_elements


def get_model_size(model, data_width=32, count_nonzero_only=False) -> int:
    """
    calculate the model size in bits
    :param data_width: #bits per element
    :param count_nonzero_only: only count nonzero weights
    """
    return get_num_parameters(model, count_nonzero_only) * data_width


print(f"Model Size: {get_model_size(model)}")

Model Size: 738016
